## 作業

1. 如何使用已經訓練好的模型？
2. 依照 https://github.com/qqwweee/keras-yolo3 的程式碼，請敘述，訓練模型時，資料集的格式是什麼？具體一點的說，要提供什麼格式的文件來描述資料集的圖片以及 bboxes 的信息呢？





In [0]:
%tensorflow_version 1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2
import tensorflow as tf
print(tf.__version__)

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.0


In [0]:
!pip install keras==2.2.4 # 需要安裝 keras 2.2.4 的版本

     |████████████████████████████████| 317kB 40.6MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
from google.colab import drive 
drive.mount('/content/gdrive') # 將 google drive 掛載在 colob，
# 下載基於 keras 的 yolov3 程式碼
%cd 'gdrive/My Drive'
# !git clone https://github.com/qqwweee/keras-yolo3 # 如果之前已經下載過就可以註解掉
%cd keras-yolo3

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive
/content/gdrive/My Drive/keras-yolo3


In [0]:
unit_test = True

In [0]:
'''
download yolo weight
'''
import os

# const value
# yolo_weight_name = 'yolov3'
yolo_weight_name = 'yolov3-tiny'

# yolo_cfg_name = 'yolov3.cfg'
yolo_cfg_name = 'yolov3-tiny_cus.cfg'

def check_yolo_weight():
    if not os.path.exists("convert.py") and not os.path.exists(yolo_cfg_name):
        print('May not in correct keras-yolo path...')
        return False

    if not os.path.exists("model_data/" + yolo_weight_name + ".h5"):
        if not os.path.exists(yolo_weight_name + ".weights"):
            print("Model doesn't exist, downloading...")
            os.system("wget https://pjreddie.com/media/files/" + yolo_weight_name + ".weights")
        
        convert_cmd = "python convert.py %s %s.weights model_data/%s.h5" % (yolo_cfg_name, yolo_weight_name, yolo_weight_name)
        os.system(convert_cmd)
        print("Converting %s.weights to %s.h5..." % (yolo_weight_name, yolo_weight_name))
    else:
        print('model is existed.')

    return True

if unit_test:
    check_yolo_weight()

model is existed.


In [0]:
'''
download dataset
'''

# const value
dataset_cfg = {
    "name" : 'VOCdevkit',
    "file_name" : 'VOCtrainval_06-Nov-2007.tar',
    "url_path" : 'http://pjreddie.com/media/files/'
}

def check_and_download_dataset(name=None, file_name=None, url_path=None):
    if name is None or file_name is None or url_path is None:
        print('Bad input of parameter...')
        return

    if not os.path.exists(name):
        os.system("wget " + url_path + file_name) # 下載 VOC 資料集
        os.system("tar xvf " + file_name) # 解壓縮資料集，會花幾分鐘
    else:
        print("data exists")


if unit_test:
    check_and_download_dataset(**dataset_cfg)

data exists


In [0]:
'''
generate list file
'''
import xml.etree.ElementTree as ET
from os import getcwd

# const value
list_train_name = '2007_train_test.txt'
list_valid_name = '2007_valid_test.txt'

# output_sets = [('2007', 'train', 100), ('2007', 'val', 20)]
# classes = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", 
#            "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", 
#            "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

classes = ["dog", "cat"]

img_path_root = 'VOCdevkit/VOC2007/'
img_id_sub_path = 'ImageSets/Main/'
jpg_sub_path = 'JPEGImages/'
annotation_sub_path = 'Annotations/'

# img_id_files = ['dog_train.txt', 'dog_val.txt', 'cat_train.txt', 'cat_val.txt']
# img_id_files = ['dog_train.txt']

img_ids_surffix = ["_train.txt", "_val.txt"]
img_id_files = [ (cls_name + surffix) for cls_name in classes for surffix in img_ids_surffix]

def convert_annotation(img_path, list_file_ptr):
    input_file = open(img_path)
    tree = ET.parse(input_file)
    root = tree.getroot()

    # search info in element tree
    for obj in root.iter('object'):
        is_difficult = int(obj.find('difficult').text)
        class_name = obj.find('name').text

        if is_difficult or class_name not in classes:
            continue
        cls_id = classes.index(class_name)
        xml_bbox = obj.find('bndbox')
        xmin = xml_bbox.find('xmin').text
        xmax = xml_bbox.find('xmax').text
        ymin = xml_bbox.find('ymin').text
        ymax = xml_bbox.find('ymax').text
        bbox_list = [xmin, ymin, xmax, ymax]
        write_str = " " + ",".join([str(int(a)) for a in bbox_list]) + ',' + str(cls_id)
        list_file_ptr.write(write_str)

def get_img_ids(id_files):
    # total train ids
    img_ids = list()

    for img_id_file in id_files:
        img_id_file_str = open(img_path_root + img_id_sub_path + img_id_file).read()
        img_id_lines = img_id_file_str.strip().split('\n')
        for id_line in img_id_lines:
            id_strs = id_line.split(' ')
            if len(id_strs) > 1:
                id_str = id_strs[0]
                id_bool = int(id_strs[-1])
                if id_bool != 1:
                    continue
            img_ids.append(id_strs[0])

    return img_ids

def generate_train_valid_file(list_train_name, list_valid_name,
                             img_id_files, train_val_ratio=0.8):
    # get img ids from file
    img_ids = get_img_ids(img_id_files)
    img_ids = list(set(img_ids))
    train_boundary = round(len(img_ids)*train_val_ratio)
    
    gen_annotation_file(list_train_name, img_ids[:train_boundary])
    gen_annotation_file(list_valid_name, img_ids[train_boundary:])

def gen_annotation_file(list_file_name, img_ids):
    with open(list_file_name, 'w') as list_file_ptr:
        for img_id in img_ids:
            # write img path
            list_file_ptr.write('./%s%s%s.jpg'%(img_path_root, jpg_sub_path, img_id))
            annotation_path = '%s%s%s.xml' % (img_path_root, annotation_sub_path, img_id)
            # write annotation info
            convert_annotation(annotation_path, list_file_ptr)
            list_file_ptr.write('\n')

if unit_test:
    if not os.path.exists(list_train_name) or not os.path.exists(list_valid_name):
        generate_train_valid_file(list_train_name, list_valid_name, img_id_files)
        print('File generate is completed.')
    else:
        print('File is existed.')


File is existed.


In [0]:
# use kmeans.py in keras-yolo
from kmeans import YOLO_Kmeans
import numpy as np

class CusYOLO_Kmeans(YOLO_Kmeans):

    def __init__(self, cluster_number, filename, output_name):
        super().__init__(cluster_number, filename)
        self.filename = filename
        self.output_name = output_name
    
    def result2txt(self, data):
        f = open(self.output_name, 'w')
        row = np.shape(data)[0]
        for i in range(row):
            if i == 0:
                x_y = "%d,%d" % (data[i][0], data[i][1])
            else:
                x_y = ", %d,%d" % (data[i][0], data[i][1])
            f.write(x_y)
        f.close()


In [0]:
# generate customer class file and anchor file

data_name = 'custom'
classes_surffix = '_classes.txt'
anchors_surffix = '_anchors.txt'
cluster_number = 9
anchors_file = 'model_data/' + data_name + anchors_surffix

def gen_classes_file(data_name):
    with open('model_data/' + data_name + classes_surffix, 'w') as fp:
        for cls_name in classes:
            fp.write(cls_name + '\n')

def gen_anchor_file(cluster_number, anchors_file):
    cus_yolo_inst = CusYOLO_Kmeans(cluster_number, list_train_name, anchors_file)
    cus_yolo_inst.txt2clusters()

if unit_test:
    gen_classes_file(data_name)
    gen_anchor_file(cluster_number, anchors_file)


K anchors:
 [[ 70  65]
 [111 142]
 [168 216]
 [211 128]
 [242 261]
 [321 436]
 [326 305]
 [392 209]
 [450 330]]
Accuracy: 77.18%


In [0]:
# Ready to train
from datetime import datetime
from train import get_classes, get_anchors, create_model, data_generator_wrapper
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


# const value
train_annotation_path = list_train_name
valid_annotation_path = list_valid_name

# log_dir = 'logs/' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_dir = 'logs/' + datetime.now().strftime("%Y-%m-%d") + '/'
classes_path = 'model_data/' + data_name + classes_surffix
anchors_path = 'model_data/' + data_name + anchors_surffix

Using TensorFlow backend.


In [0]:

input_shape = (416,416)
batch_size = 8
epochs_count = 50
epochs_count = 200
initial_epoch = 0
initial_epoch = 50

learning_rate = 1e-3
# learning_rate = 1e-4

yolo_base_model_path = 'model_data/' + yolo_weight_name +'.h5'
yolo_base_model_path = 'logs/2020-02-24/custom_weights_2020-02-24_18-01-07.h5'

def get_annotation_lines(annotation_path):
    with open(annotation_path) as fp:
        ann_lines = fp.readlines()
    np.random.shuffle(ann_lines)
    
    return ann_lines


def train_with_data(classes_path, anchors_path, input_shape=(416, 416), is_trainable=True):
    class_names = get_classes(classes_path)
    num_classes = len(class_names)
    anchors = get_anchors(anchors_path)
    # load yolo pretrain weight
    model = create_model(input_shape, anchors, num_classes, freeze_body=0, weights_path=yolo_base_model_path)
    
    # log initialize and related setting
    if not os.path.exists(log_dir):
        os.system("mkdir -p " + log_dir)

    logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1)

    train_ann_lines = get_annotation_lines(train_annotation_path)
    valid_ann_lines = get_annotation_lines(valid_annotation_path)

    num_train = len(train_ann_lines)
    num_val = len(valid_ann_lines) 

    callbacks_list = [logging, checkpoint]
    if is_trainable:
        callbacks_list.extend([reduce_lr, early_stopping])
        # callbacks_list.extend([reduce_lr])
        for i in range(len(model.layers)):
            model.layers[i].trainable = True

    model.compile(optimizer=Adam(lr=learning_rate), loss={'yolo_loss': lambda y_true, y_pred: y_pred})

    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    model.fit_generator(data_generator_wrapper(train_ann_lines, batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(valid_ann_lines, batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=epochs_count,
            initial_epoch=initial_epoch,
            callbacks=callbacks_list)
    model_file_name = 'custom_weights_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.h5'
    model.save_weights(log_dir + model_file_name)
    
    return model_file_name

if unit_test:
    model_file_name = train_with_data(classes_path, anchors_path, input_shape=input_shape)
    pass
















Create YOLOv3 model with 9 anchors and 2 classes.
Load weights logs/2020-02-24/custom_weights_2020-02-24_18-01-07.h5.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 600 samples, val on 150 samples, with batch size 8.




Epoch 51/200
75/75 [==============================] - 329s 4s/step - loss: 21.2319 - val_loss: 184.9337

Epoch 52/200
75/75 [==============================] - 71s 944ms/step - loss: 20.4776 - val_loss: 17202.1033
Epoch 53/200
75/75 [==============================] - 71s 945ms/step - loss: 19.4792 - val_loss: 21.7161
Epoch 54/200
75/75 [==============================] - 71s 943ms/step - loss: 19.0326 - val_loss: 20.0357
Epoch 55/200
75/75 [==============================] - 71s 948ms/step - loss: 18.4894 - val_loss: 19.1148
Epoch 56/200
75/75 [==============================] - 71s 948ms/step - loss: 18.1813 - val_loss: 18.7158
Epoch 57/200
75/75 [==============================] - 71s 950ms/step - loss

# New Section

In [0]:
# from yolo import YOLO
# from train import get_classes
# classes = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", 
#            "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", 
#            "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

# classes_path = 'model_data/voc_classes.txt'
# class_names = get_classes(classes_path)
# yolo_model = YOLO(model_path='model_data/yolov3.h5', classes_path=classes_path)

# cat_img = Image.open('../test_img/001185.jpg')

# r_image = yolo_model.detect_image(cat_img)
# r_image

In [0]:
from yolo import YOLO
log_dir = 'logs/2020-02-25/'
# yolo_model = YOLO(model_path=log_dir + model_file_name, classes_path=classes_path)
yolo_model = YOLO(model_path=log_dir + 'custom_weights_2020-02-25_07-17-58.h5', classes_path=classes_path)
# yolo_model = YOLO(model_path=log_dir + 'custom_weights_2020-02-23_05-23-05.h5', classes_path=classes_path)
# yolo_model = YOLO(model_path=log_dir + 'custom_weights_2020-02-19_03-06-14.h5', classes_path=classes_path)

logs/2020-02-25/custom_weights_2020-02-25_07-17-58.h5 model, anchors, and classes loaded.


In [0]:
from PIL import Image
import matplotlib.pyplot as plt

img_path_root = 'VOCdevkit/VOC2007/'
img_id_sub_path = 'ImageSets/Main/'
jpg_sub_path = 'JPEGImages/'
annotation_sub_path = 'Annotations/'

images_path = ['../test_img/001185.jpg', '../test_img/dog2.jpeg', 
               img_path_root + jpg_sub_path + '007585.jpg',
               img_path_root + jpg_sub_path + '007334.jpg', '../test_img/cats.jpg']


def show_images(images, is_bgr=False):
    plt.figure(figsize=(20,20))
    for img_idx, tmp_img in enumerate(images):
        print(type(tmp_img))
        
        tmp_img = np.asarray(tmp_img)
        if is_bgr:
            tmp_img = tmp_img[:,:,::-1]
        print(tmp_img.shape)
        plt.subplot(331+img_idx)
        plt.imshow(tmp_img)

    plt.tight_layout()
    plt.show()

def detect_images(yolo_model, images_path):
    predict_imgs = list()
    for image_path in images_path:
        tmp_img = Image.open(image_path)
        tmp_predict_img = yolo_model.detect_image(tmp_img)
        predict_imgs.append(tmp_predict_img)

    return predict_imgs

if unit_test:
    predict_imgs = detect_images(yolo_model, images_path)
    show_images(predict_imgs)
    pass


Output hidden; open in https://colab.research.google.com to view.

In [0]:
# r_image